In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
import xgboost
from gmdhpy import gmdh 
from sklearn.model_selection import RandomizedSearchCV
import pickle

In [2]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join('/home/osboxes/repos/sport/competitions_sendy/data/raw/SampleSubmission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='Order_No')
    
    submission['Time from Pickup to Arrival'] = prediction
    submission.to_csv(filename)

In [3]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [4]:
train = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/train_2.csv', sep=',') 

In [5]:
test = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/test_2.csv', sep=',') 

In [19]:
#list(zip(train.columns, test.columns))

In [7]:
target = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/target_2.csv', sep=',') 

In [8]:
y_train = target['Time from Pickup to Arrival']

In [22]:
#train.shape, test.shape

In [27]:
train_cols = list(train.columns[1:])

In [28]:
train_cols

['No_Of_Orders',
 'Age',
 'Average_Rating',
 'No_of_Ratings',
 'Platform Type',
 'Pickup - Day of Month',
 'Pickup - Weekday (Mo = 1)',
 'Arrival at Pickup - Day of Month',
 'Arrival at Pickup - Weekday (Mo = 1)',
 'Distance (KM)',
 'Temperature',
 'Pickup Lat',
 'Pickup Long',
 'Destination Lat',
 'Destination Long',
 'Precipitation',
 'pickup__hhh',
 'pickup__nn',
 'pickup__ss',
 'pickup__timeall',
 'arrival_pickup__hhh',
 'arrival_pickup__nn',
 'arrival_pickup__ss',
 'arrival_pickup__timeall',
 'dist_2',
 'time_2',
 'speed_2']

# Simple

In [12]:
test['time_2'].values

array([1493. , 1153.8,  787.4, ..., 1167.5, 3397.5, 1704.9])

In [13]:
write_submission_file(prediction=test['time_2'].values, filename='/home/osboxes/repos/sport/competitions_sendy/submissions/test_1.csv')

# Boosting + opt

In [29]:
params = {
    "learning_rate"    : [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.15, 0.20, 0.25, 0.30 ],
    "max_depth"        : list(range(1,16)),
    "min_child_weight" : list(range(1,10)),
    "gamma"            : [0.5, 1, 1.5, 2, 5],
    "subsample"        : [0.6, 0.8, 1.0],
    "colsample_bytree" : [0.6, 0.8, 1.0],       
    "n_estimators"     : [100, 200, 300, 400, 500, 600, 700, 800, 1000, 1500, 2000],
    "seed"             : [42]
}

In [30]:
estimator = xgboost.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')

In [31]:
param_comb = 15
clf = RandomizedSearchCV(estimator, param_distributions=params, n_iter=param_comb, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=2, random_state=1)

In [32]:
%%time
clf.fit(train[train_cols], y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 26.0min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 27.0min finished
/home/osboxes/envs/sport/lib64/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/osboxes/envs/sport/lib64/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


CPU times: user 1min 8s, sys: 137 ms, total: 1min 8s
Wall time: 28min 10s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          eval_metric='rmse', gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='re...
                                                          0.04, 0.05, 0.06,
                                                          0.07, 0.08, 0.09, 0.1,
                                

In [33]:
clf.best_score_, clf.best_params_

(-563971.7110202888,
 {'subsample': 1.0,
  'seed': 42,
  'n_estimators': 800,
  'min_child_weight': 9,
  'max_depth': 7,
  'learning_rate': 0.03,
  'gamma': 1,
  'colsample_bytree': 0.6})

In [34]:
model = xgboost.XGBRegressor(**clf.best_params_)
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=1,
             importance_type='gain', learning_rate=0.03, max_delta_step=0,
             max_depth=7, min_child_weight=9, missing=None, n_estimators=800,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=1.0, verbosity=1)

In [35]:
%%time
model.fit(train[train_cols], y_train)
tree_prediction = model.predict(test[train_cols])

[16:26:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 1min 7s, sys: 112 ms, total: 1min 8s
Wall time: 1min 8s


In [36]:
write_submission_file(prediction=tree_prediction, filename='/home/osboxes/repos/sport/competitions_sendy/submissions/xgb_new_1.csv')